# Probabilistic models


Now, we will see some probabilistic models (without adding extra features): naive approach, moving average, ARIMA, and Prophet

In [1]:
import pandas as pd
import numpy as np
import joblib
from tqdm.notebook import tqdm as tqdm


from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARIMA
from prophet import Prophet

import warnings
warnings.filterwarnings("ignore")

c:\Users\agost\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Download csv file from resources and put it in working directory

calendar = pd.read_csv('../Demand Forecasting M5/calendar.csv', header=0)
sales = pd.read_csv(
    '../Demand Forecasting M5/sales_train_evaluation.csv', header=0)
sales_val = pd.read_csv(
    '../Demand Forecasting M5/sales_train_validation.csv', header=0)
sample_sub = pd.read_csv(
    '../Demand Forecasting M5/sample_submission.csv', header=0)
sell_prices = pd.read_csv('../Demand Forecasting M5/sell_prices.csv', header=0)


## Train/Val split 

First, we need to create training and validation sets to train and validate our models. We will use the last 28 days sales as  validation data and the sales of the previous days as training data.

In [3]:
d_cols_sales = [c for c in sales.columns if 'd_' in c]
d_cols_val = [c for c in sales_val.columns if 'd_' in c]

In [4]:
scale_dataset = 0.15 # Parameter that decides how many last samples we want to consider
val_size = 28 # We have to predict the next 28 days
test_size = 28 # Dimension of the test set

In [5]:
train_size_max = len(d_cols_val) - val_size # The maximum length that the train set can have

# Take a multiple of 28 days (and we want at least 28 elements)
train_size = max(val_size, int(((train_size_max*scale_dataset)//val_size)*val_size)) # Size of the train set

print("You have choosen " + str(train_size) + " elements for the train set")

total_train_val = train_size + val_size 
total_train_val_test = total_train_val + test_size


You have choosen 280 elements for the train set


In [6]:
train_dataset = sales_val[d_cols_val[-total_train_val:-val_size]]
val_dataset = sales_val[d_cols_val[-val_size:]]
train_val_dataset = sales[d_cols_sales[-total_train_val-test_size:-test_size]]
test_dataset = sales[d_cols_sales[-test_size:]]

In [7]:
train_val_dataset.head()

,d_1606,d_1607,d_1608,d_1609,d_1610,d_1611,d_1612,d_1613,d_1614,d_1615,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,1,1,0,0,0,1,1,2,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,0,1,0,0,...,2,1,2,1,1,1,0,1,1,1
3,3,1,1,4,1,4,2,2,0,2,...,1,0,5,4,1,0,1,3,7,2
4,0,1,1,0,0,3,3,1,1,0,...,2,1,1,0,1,1,2,2,2,4


In [8]:
train_dataset.head()

,d_1606,d_1607,d_1608,d_1609,d_1610,d_1611,d_1612,d_1613,d_1614,d_1615,...,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885
0,0,1,1,0,0,0,1,1,2,0,...,3,1,3,1,2,2,0,1,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
2,0,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,1,1,0
3,3,1,1,4,1,4,2,2,0,2,...,4,2,1,4,1,3,5,0,6,6
4,0,1,1,0,0,3,3,1,1,0,...,3,2,2,2,3,1,0,0,0,0


In [9]:
val_dataset.head()

,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,1,0,0,0,0,0,1,0,4,2,...,1,3,0,1,1,1,3,0,1,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,3,1,2,1,3,1,...,1,0,5,4,1,0,1,3,7,2
4,1,0,4,4,0,1,4,0,1,0,...,2,1,1,0,1,1,2,2,2,4


In [10]:
test_dataset.head()

,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0,0,0,2,0,3,5,0,0,1,...,2,4,0,0,0,0,3,3,0,1
1,0,1,0,0,0,0,0,0,0,1,...,0,1,2,1,1,0,0,0,0,0
2,0,0,1,1,0,2,1,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,0,0,1,2,4,1,6,4,0,0,...,1,1,0,4,0,1,3,0,2,6
4,1,0,2,3,1,0,3,2,3,1,...,0,0,0,2,1,0,0,2,1,0


In [11]:
# Plot of three samples

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(train_size), mode='lines', y=train_dataset.loc[0].values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(train_size, total_train_val), y=val_dataset.loc[0].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(train_size), mode='lines', y=train_dataset.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(train_size, total_train_val), y=val_dataset.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(train_size), mode='lines', y=train_dataset.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(train_size, total_train_val), y=val_dataset.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Train (blue) vs. Validation (orange) sales")
fig.show()

In [12]:
# Define function to plot predictions

def plot_prediction(pred_0, pred_1, pred_2, title_text):


    fig = make_subplots(rows=3, cols=1)

    fig.add_trace(
        go.Scatter(x=np.arange(train_size), mode='lines', y=train_dataset.loc[0].values, marker=dict(color="dodgerblue"),
                name="Train"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=np.arange(train_size, total_train_val), y=val_dataset.loc[0].values, mode='lines', marker=dict(color="darkorange"),
                name="Val"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=np.arange(train_size, total_train_val), y=pred_0, mode='lines', marker=dict(color="seagreen"),
                name="Prediction"),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=np.arange(train_size), mode='lines', y=train_dataset.loc[1].values, marker=dict(color="dodgerblue"), showlegend=False,
                name="Train"),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=np.arange(train_size, total_train_val), y=val_dataset.loc[1].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
                    name="Val"),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=np.arange(train_size, total_train_val), y=pred_1, mode='lines', marker=dict(color="seagreen"), showlegend=False,
                name="Prediction"),
        row=2, col=1
    )

    fig.add_trace(
        go.Scatter(x=np.arange(train_size), mode='lines', y=train_dataset.loc[2].values, marker=dict(color="dodgerblue"), showlegend=False,
                name="Train"),
        row=3, col=1
    )

    fig.add_trace(
        go.Scatter(x=np.arange(train_size, total_train_val), y=val_dataset.loc[2].values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
                    name="Val"),
        row=3, col=1
    )

    fig.add_trace(
        go.Scatter(x=np.arange(train_size, total_train_val), y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
                name="Prediction"),
        row=3, col=1
    )

    fig.update_layout(height=1200, width=800, title_text= title_text)
    fig.show()


In [13]:
# Define metrics

def mae(pred, value):
    return (np.abs(pred - value)).mean()
    
def rmse(pred, value):
    return np.sqrt(((pred-value)**2).mean())


## Naive approach 


The first approach is the very simple naive approach. It simply forecasts the next day's sales as the current day's sale.

In practice the prediction of the model for today (yt), is the predicted value for the next day (yt-1). 
The model predicts tomorrow's sales as today's sales. 

In [14]:
predictions_naive = []

for i in range(val_size):
    if i == 0:
        predictions_naive.append(train_dataset[train_dataset.columns[-1]].values) # takes the last value (column) of the train_dataset
    else:
        predictions_naive.append(val_dataset[val_dataset.columns[i-1]].values) # takes the previous value as the actual value

predictions_naive = np.transpose(np.array(predictions_naive))

# Root mean squared error
error_naive_rmse = rmse(predictions_naive[:3],val_dataset.values[:3])
# Mean absolute error
error_naive_mae = mae(predictions_naive[:3],val_dataset.values[:3])

print(error_naive_rmse)

0.9880235200593537


In [15]:
pred_0 = predictions_naive[0]
pred_1 = predictions_naive[1]
pred_2 = predictions_naive[2]

plot_prediction(pred_0, pred_1, pred_2, title_text = "Naive approach")

We can see that the forecasts made by the naive approach are not accurate and it is to be expected of such a simple algorithm. We need more complex models which use several time stamps to make forecasts.

## Moving average 


The moving average method is more complex than the naive approach. It calculates the mean sales over the previous 28 (or any other number) days and forecasts that as the next day's sales. This method takes the previous 30 timesteps into consideration, and is therefore less prone to short term fluctuations than the naive approach.

In [16]:
def moving_avg(data, val, window_size):
  predictions_ma = []

  for i in range(val_size):
      if i == 0:
          predictions_ma.append(np.mean(data[data.columns[-window_size:]].values, axis=1))

      if i < val_size+1 and i > 0:
          predictions_ma.append((np.mean(data[data.columns[-window_size+i:]].values, axis=1) + np.mean(predictions_ma[:i], axis=0))*0.5)
      if i > val_size+1:
          predictions_ma.append(np.mean([predictions_ma[:i]], axis=1))
      
  predictions_ma = np.transpose(np.array(predictions_ma))
  error_mae = mae(predictions_ma,val.values)
  error_ma_rmse = rmse(predictions_ma, val.values)
  print(error_ma_rmse)
  return predictions_ma, error_ma_rmse



window=[7,14,21,28,35,42,49,60,70,80,90]
for i in window:
  print("rmse with window "+ str(i) + ":")
  moving_avg(train_dataset[:3], val_dataset[:3], i)


rmse with window 7:
0.8386371198601981
rmse with window 14:
0.8277064495154398
rmse with window 21:
0.8062396330766359
rmse with window 28:
0.81890807473352
rmse with window 35:
0.8082920578853177
rmse with window 42:
0.7853943612791099
rmse with window 49:
0.7797104879657559
rmse with window 60:
0.7803473723056946
rmse with window 70:
0.7752269158661534
rmse with window 80:
0.7789010234571131
rmse with window 90:
0.7829413768219872


In [17]:
predictions_ma,error_ma_rmse = moving_avg(train_dataset[:3], val_dataset[:3], 70)

0.7752269158661534


In [18]:
#window=[7,14,21,28,35,42,49,60,70,80,90]
#for i in window:
#  print("rmse with window "+ str(i) + ":")
#  predictions_ma_test, error_ma_rmse_test = moving_avg(train_val_dataset[:3], test_dataset[:3],i)

In [19]:
pred_0 = predictions_ma[0]
pred_1 = predictions_ma[1]
pred_2 = predictions_ma[2]
plot_prediction(pred_0, pred_1, pred_2, title_text = "Moving Average")

We can see that this model performs better than the naive approach. It is less susceptible to the volatility in day-to-day sales data and manages to pick up trends with slightly higher accuracy. However, it is still unable to find high-level trends in the sales.

## ARIMA 

ARIMA stands for Auto Regressive Integrated Moving Average. ARIMA models aims at describing the correlations in the time series.


In [20]:
predictions_arima = []
for row in tqdm(train_dataset[train_dataset.columns[-val_size:]].values[:3]):
    fit = sm.tsa.statespace.SARIMAX(row, seasonal_order=(0, 1, 1, 7)).fit()
    predictions_arima.append(fit.forecast(val_size))
predictions_arima = np.array(predictions_arima).reshape((-1, val_size))
error_arima_rmse = rmse(predictions_arima[:3],val_dataset.values[:3])
print(error_arima_rmse)


  0%|          | 0/3 [00:00<?, ?it/s]

0.9365861694123355


In [21]:
pred_0 = predictions_arima[0]
pred_1 = predictions_arima[1]
pred_2 = predictions_arima[2]

plot_prediction(pred_0, pred_1, pred_2, title_text = "ARIMA")

ARIMA is able to find low-level and high-level trends simultaneously, unlike most other models which can only find one of these. It is able to predict a periodic function for each sample, and these functions seem to be pretty accurate (except for the second sample).

We can tune ARIMA model based on singolar store, because here we don't obtain good results.

# Prophet 


Prophet is an opensource time series forecasting project by Facebook. It is based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, including holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. It is also supposed to be more robust to missing data and shifts in trend compared to other models.

In [22]:

cal_train = calendar.date[1942-train_size -val_size-test_size: 1942 -val_size -test_size]
cal_val = calendar.date[1942-val_size-test_size: 1942-test_size]
#print(cal_train)
#print(cal_val)


dates = list(cal_val)

predictions_prophet = []

for row in tqdm(train_dataset[train_dataset.columns[-val_size:]].values[:3]):
    df = pd.DataFrame(np.transpose([dates, row]))

    df.columns = ["ds", "y"] # dates and the y value

    model = Prophet(daily_seasonality=True, seasonality_mode = 'multiplicative')
    model.fit(df)
    future = model.make_future_dataframe(periods=val_size)
    forecast = model.predict(future)["yhat"].loc[val_size:].values

    predictions_prophet.append(forecast)
predictions_prophet = np.array(predictions_prophet).reshape((-1, val_size))
error_prophet_rmse = rmse(predictions_prophet[:3], val_dataset.values[:3])
print(error_prophet_rmse)

  0%|          | 0/3 [00:00<?, ?it/s]

15:11:51 - cmdstanpy - INFO - Chain [1] start processing
15:11:51 - cmdstanpy - INFO - Chain [1] done processing
15:11:51 - cmdstanpy - INFO - Chain [1] start processing
15:11:51 - cmdstanpy - INFO - Chain [1] done processing
15:11:52 - cmdstanpy - INFO - Chain [1] start processing
15:11:52 - cmdstanpy - INFO - Chain [1] done processing


1.1715994804045688


In [23]:
pred_0 = predictions_prophet[0]
pred_1 = predictions_prophet[1]
pred_2 = predictions_prophet[2]

plot_prediction(pred_0, pred_1, pred_2, title_text = "Prophet")

Prophet appears to output very similar-shaped predictions to ARIMA. But on closer observation, we can see that the there is a macroscopic upward trend which was absent in ARIMA. In the ARIMA predictions, the exact same pattern was repeated. But in the Prophet predictions, the same pattern is shifted vertically at each oscillation. This shows that is able to capture high-level trends better than ARIMA.


## Loss for each model 

In [24]:
error = [error_naive_rmse, error_ma_rmse, error_arima_rmse, error_prophet_rmse]

names = ["Naive approach", "Moving average", "ARIMA", "Prophet"]
df = pd.DataFrame(list(zip(error, names)), columns =["RMSE Loss", "Model"])


px.bar(df, y="RMSE Loss", x="Model", color = "Model", title="RMSE Loss vs. Model")

From the above graph, we can see that the moving average method perform better than the others. The remaining models: naive approach, ARIMA, and Prophet are the worst-scoring models. We think that the accuracy of ARIMA and Prophet can be boosted significantly by tuning the hyperparameters.